In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf

In [2]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential

In [3]:
model = Sequential([
    Conv2D(512, (3,3), input_shape=(48,48,1),activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(512,(3,3), activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(256,(3,3), activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(256,(3,3), activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Flatten(),
    Dense(512,activation = 'relu'),
    Dropout(0.2),
    Dense(256,activation = 'relu'),
    Dense(7,activation = 'softmax')
])

In [4]:
model.load_weights('kdef_gray_model.h5')

## Loading a test image and preprocessing it

In [5]:
test_image = cv2.imread('testimage_cropped.jpg', cv2.IMREAD_GRAYSCALE)

In [6]:
cv2.imshow('display', test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
test_image_resized_gray = cv2.resize(test_image,(48,48))

In [8]:
cv2.imshow('display', test_image_resized_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Prediction!

In [9]:
def predict_emotion(img):
    img = cv2.resize(img, (48,48))
    prediction = model.predict(img.reshape(1,48,48,1))
    prediction = np.argmax(prediction, axis = -1)
    emotions = ['ANGRY','FEARFUL','DISGUSTED','HAPPY','SAD','SURPRISED','NEUTRAL']
    
    return emotions[int(prediction)]

In [10]:
predict_emotion(test_image_resized_gray)

'DISGUSTED'

In [11]:
img = cv2.imread('./test/happy/PrivateTest_13103594.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (48,48))

In [12]:
img.shape

(48, 48)

In [13]:
cv2.imshow('Test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
predict_emotion(img)

'DISGUSTED'

## Using HC to create a bounding box around face found in image

In [15]:
hc_path = '.\haarcascade_frontalface_default.xml'

In [16]:
hc = cv2.CascadeClassifier(hc_path)

In [17]:
faces = hc.detectMultiScale(test_image,minNeighbors = 3,scaleFactor = 1.2)

In [18]:
adjust_param = [0,0]

In [19]:
for face in faces:
    x, y, w, h = face
    cv2.rectangle(test_image,(x - adjust_param[0],y - adjust_param[1]),(x+w +adjust_param[0], y+h + adjust_param[1]),(0,0,255),3)

In [20]:
def draw_bounding_box(img):
    faces = hc.detectMultiScale(img)
    for face in faces:
        x, y, w, h = face
        cv2.rectangle(img,(x - adjust_param[0],y - adjust_param[1]),(x+w +adjust_param[0], y+h + adjust_param[1]),(0,0,255),3)
    return img

In [21]:
cv2.imshow("Faces Detected", test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
cv2.imshow("Faces Detected", draw_bounding_box(test_image))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
adjust_param = [50,50]

In [24]:
for face in faces:
    x, y, w, h = face
    cv2.rectangle(test_image,(x - adjust_param[0],y - adjust_param[1]),(x+w +adjust_param[0], y+h + adjust_param[1]),(0,0,255),3)

In [25]:
def draw_bounding_box(img):
    faces = hc.detectMultiScale(img)
    for face in faces:
        x, y, w, h = face
        cv2.rectangle(img,(x - adjust_param[0],y - adjust_param[1]),(x+w +adjust_param[0], y+h + adjust_param[1]),(0,0,255),3)
    return img

In [26]:
cv2.imshow("Faces Detected", test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
cv2.imshow("Faces Detected", draw_bounding_box(test_image))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
x, y, w, h = faces[0]
cropped_face = test_image[y:y+h, x:x+w]

In [29]:
def return_cropped_face(img):
    faces = hc.detectMultiScale(img)
    for (x,y,w,h) in faces:
        img = img[y:y+h, x:x+w]
    return img

In [30]:
cv2.imshow("Cropped Face", cropped_face)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
predict_emotion(cropped_face)

'DISGUSTED'

In [32]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
video_cap = cv2.VideoCapture(0)

In [ ]:
if not video_cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = video_cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = hc.detectMultiScale(gray)
    
    for (x,y,w,h) in faces:
        face = gray[y:y+h, x:x+w]
        
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, 
            predict_emotion(face), 
            (50, 50), #top left
            font, 1, 
            (255, 0, 0), #BGR 
            2, 
            cv2.LINE_4)
    # Display the resulting frame
    cv2.imshow('frame', draw_bounding_box(frame))
    if cv2.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
video_cap.release()
cv2.destroyAllWindows()